<a href="https://colab.research.google.com/github/juanchess98/Notebooks-Deep-Learning/blob/Recurrent-Neural-Network/Recurrent_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Redes Neuronales Recurrentes - Recurrent Neural Networks (RNN's)
Una red neuronal recurrente se parece mucho a una red neuronal convencional con la excepción de que también tiene conexiones apuntando hacia atrás.
La RNN más simple está compuesta de una neurona que recibe entradas y produce una salida y envía la salida de vuelta a si misma. En cada time step $t$   (también llamado frame), esta neurona recurrente recibe las entradas $\mathbf{x}_{(t)}$ así como su propio output del time step anterior, $\mathbf{y}_{(t-1)}$.
Ya que no hay un output anterior en el primer time step, este se asigna generalmente el valor de 0. Esta pequeña red se puede representar contra el eje del tiempo. Esto se conoce como desenrrollar la red en el tiempo(unrolling the network through time).
![RNN](https://drive.google.com/uc?id=1zOI-isp2DN-42ZFjtBKK1XFXDkSi3FiM)

Tu puedes crear facilmente un layer de neuronas recurrentes. En cada time step $t$, cada neurona recibe el vector de entrada $\mathbf{x}_{(t)}$ y el output del vector del time step anterior $\mathbf{y}_{(t-1)}$. Note que las entradas y las salidas son vector ! 

Cada neurona recurrente tiene dos pesos. Uno para las entradas $\mathbf{x}_{(t)}$ y otro para las salidas del time step anterior $\mathbf{y}_{(t-1)}$ denotados como las matrices $\mathbf{W}_{x}$ y $\mathbf{W}_{y}$ respectivamente.

La salida de una capa recurrente para una sola instancia está dada así:
$$\mathbf{y}_{(t)} = \phi(\mathbf{W}_{x}^{T}\mathbf{x}_{(t)} +\mathbf{W}_{y}^{T}\mathbf{y}_{(t-1)} + \mathbf{b})$$
$\mathbf{b}$ es el vector bias y $\phi(.)$ es la función de activación(ReLu o tanh son las más utilizadas).

Asi como en las redes neuronales vistas anteriormente, se puede computar la salida de una capa de una capa recurrente de una sola vez al colocar todas las entradas en el time step t en una matriz de entrada $\mathbf{X}_{(t)}$ así:

$$\mathbf{Y}_{(t)} = \phi(\mathbf{W}_{x}^{T}\mathbf{X}_{(t)} +\mathbf{W}_{y}^{T}\mathbf{Y}_{(t-1)} + \mathbf{b})$$



## Memory Cells

Ya que la salida de una neurona recurrente en cada time step es una función de todas las entradas de los time step anteriores se podría decir que tiene una forma de memoria. Una parte de una red neuronal que preserva algún tipo de estado a lo largo de los time steps es llamada una memory cell(o simplemente una cell). A una simple neurona recurrente o una capa de neuronas recurrentes es una cell muy básica, capaz de de aprender patrones cortos(tipicamente de un largo de 10 steps, pero varía de acuerdo a la tarea).

De manera general, el estado de una celda en el time step $t$, denotado como $\mathbf{h}_{(t)}$(la "h" es por "hidden"), es una función de algunas entradas en ese time step y de su estado en el time step anterior: $\mathbf{h}_{(t)} = f(\mathbf{h}_{(t-)}, \mathbf{x}_{(t)})$. Su salida en el time step $t$, denotada como $\mathbf{y}_{(t)}$ es también una función del estado previo y las entradas del time step actuales. En el caso las celdas básicas que hemos discutido hasta ahora la salida es simplemente igual al estado, pero en celdas más complejas este no es siempre el caso. La salida en este caso es: $\mathbf{Y}_{(t)} = \mathbf{W}_{ht}\mathbf{h}_{(t)}$ una matriz de pesos que multiplica al estado actual. 

![RNN with output different](https://drive.google.com/uc?id=1dGUf3itlQnss6TkQ3sR9W6q-1oDtoZDY)
